# GNN code

In [ ]:
# <gnn environment setting>

# wget https://repo.anaconda.com/archive/Anaconda3-2024.02-1-Linux-x86_64.sh

# ls #설치 확인

# bash Anaconda3-2024.02-1-Linux-x86_64.sh  # enter 후, 마지막에 yes

# source ~/.bashrc

# conda list 

# conda create -n main python=3.10

# conda activate main

# conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia


# pip install vllm
# pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org pandas
# pip install langchain
# pip install langchain-community
# pip install langchain-openai
# /usr/bin/python -m pip install --upgrade pip
# pip install --upgrade transformers
# pip3 install openpyxl
# pip install git+https://github.com/huggingface/peft
# pip install scikit-learn
# pip install bitsandbytes
# pip install pandas
# pip install torch_geometric

# pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
# /usr/bin/python -m pip install --upgrade pip
# pip install --upgrade transformers
# pip3 install openpyxl
# pip install git+https://github.com/huggingface/peft

In [ ]:
import pandas as pd
import torch, gc
import torch
import numpy as np
import random
import os

In [ ]:
gc.collect()  #메모리 관리

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.empty_cache()

# 데이터 로드

In [ ]:
class config():
    def __init__(self, i):
        self.i = i
        self.train_edge_file_path = f'/home/work/lib_data/nghl/kisti/data/0910/240910_18_full_edge(3년후).csv'
        self.train_node_file_path = f'/home/work/lib_data/nghl/kisti/data/0910/240910_18_full_node(3년후).csv'
        self.test_edge_file_path = f'/home/work/lib_data/nghl/kisti/data/0910/240910_19_full_edge(3년후).csv' 
        self.test_node_file_path = f'/home/work/lib_data/nghl/kisti/data/0910/240910_19_full_node(3년후).csv'
        
        self.label_col=25   #입력변수 개수
        self.batch_size=1
        self.in_dim=25  #col 개수로 맞추어 주기(상동)
        self.h_dim=64
        self.num_layers=6
        self.out_dim=1
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
seed_everything(42)

In [ ]:
#train
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()  # 노드에 적용
scaler2 = RobustScaler()
start=2018

# 딕셔너리를 사용하여 edge 데이터를 저장
def train_dic():
    edges = {}
    edges_attr = {}
    nodes = {}
    labels = {}

    for i in range(1):
        cfg=config(i)
        edge_file_path = cfg.train_edge_file_path
        node_file_path = cfg.train_node_file_path
        print(edge_file_path)
        print(node_file_path) 
        
        edges[f'edge{i}'] = pd.read_csv(edge_file_path)
        nodes[f'node{i}'] = pd.read_csv(node_file_path)

        labels[f'label{i}'] = pd.DataFrame(nodes[f'node{i}'][['3년매출액증가율']]).apply(pd.to_numeric, errors='coerce')    #label 지정
        nodes[f'node{i}'] = nodes[f'node{i}'].drop(columns=['br_num_year', '3년매출액증가율']).apply(pd.to_numeric, errors='coerce')   #drop할 columns 지정
        
        nodes[f'node{i}'] = scaler.fit_transform(nodes[f'node{i}'])
        
        edges_attr[f'edge_attr{i}'] = pd.DataFrame(edges[f'edge{i}'].iloc[2,:]).apply(pd.to_numeric, errors='coerce')
        edges_attr[f'edge_attr{i}'] = scaler2.fit_transform(edges_attr[f'edge_attr{i}'])

        edges[f'edge{i}'] = edges[f'edge{i}'].iloc[:2,:].astype(int)

        labels[f'label{i}'] = labels[f'label{i}'].applymap(lambda x: 1 if x >= 72.8 else 0)  #매출액 증가율 기반 label 설정(예측 년도 유망성)
        
    return edges, edges_attr, nodes, labels

In [ ]:
#데이터 타입 변환
edges, edges_attr, nodes, labels = train_dic()

for i in range(1):
    edges[f'edge{i}'] = edges[f'edge{i}'].values.astype(np.int64)
    edges[f'edge{i}'] = torch.from_numpy(edges[f'edge{i}']).long()

    nodes[f'node{i}'] = torch.from_numpy(nodes[f'node{i}']).float()
    
    labels[f'label{i}'] = labels[f'label{i}'].values
    labels[f'label{i}'] = torch.from_numpy(labels[f'label{i}']).float()  
    
    edges_attr[f'edge_attr{i}'] = torch.from_numpy(edges_attr[f'edge_attr{i}']).float() 

# 추론용

In [ ]:
#test 차년도
# 딕셔너리를 사용하여 edge 데이터를 저장

def test_dic2():
    testedges = {}
    testedges_attr={}
    testnodes = {}
    testlabels = {}

    for i in range(1):
        cfg=config(i)
        edge_file_path = cfg.test_edge_file_path 
        node_file_path = cfg.test_node_file_path
        print(edge_file_path)
        print(node_file_path)        
        testedges[f'edge{i}'] = pd.read_csv(edge_file_path)
        testnodes[f'node{i}'] = pd.read_csv(node_file_path)
        testlabels[f'label{i}'] = pd.DataFrame(testnodes[f'node{i}'][['3년매출액증가율']]).apply(pd.to_numeric, errors='coerce')    #label 지정
        testnodes[f'node{i}'] = testnodes[f'node{i}'].drop(columns=['br_num_year','3년매출액증가율']).apply(pd.to_numeric, errors='coerce')    #drop할 columns 지정
        
        testnodes[f'node{i}'] = scaler.transform(testnodes[f'node{i}'])

        testedges_attr[f'edge_attr{i}'] = pd.DataFrame(testedges[f'edge{i}'].iloc[2,:]).apply(pd.to_numeric, errors='coerce')
        testedges_attr[f'edge_attr{i}'] = scaler2.transform(testedges_attr[f'edge_attr{i}'])
        
        testedges[f'edge{i}'] = testedges[f'edge{i}'].iloc[:2,:].astype(int)
        
        testlabels[f'label{i}'] = testlabels[f'label{i}'].applymap(lambda x: 1 if x >= 72.8 else 0)   #매출액 증가율 기반 label 설정(예측 년도 유망성)
        
    return testedges,testedges_attr,testnodes,testlabels

In [ ]:
#데이터 타입 변환
testedges2,testedges_attr2,testnodes2,testlabels2 = test_dic2()

for i in range(1):
    testedges2[f'edge{i}'] = testedges2[f'edge{i}'].values.astype(np.int64)
    testedges2[f'edge{i}'] = torch.from_numpy(testedges2[f'edge{i}']).long()

    testnodes2[f'node{i}'] = torch.from_numpy(testnodes2[f'node{i}']).float()
    
    testlabels2[f'label{i}'] = testlabels2[f'label{i}'].values
    testlabels2[f'label{i}'] = torch.from_numpy(testlabels2[f'label{i}']).float()
      
    testedges_attr2[f'edge_attr{i}'] = torch.from_numpy(testedges_attr2[f'edge_attr{i}']).float()    

# Dataloader

In [ ]:
#학습용/테스트용 그래프 데이터 준비
import torch_geometric
from torch_geometric.data import Data

def uselst():
    lst=[]
    for i in range(1):
        lst.append(Data(x=nodes[f'node{i}'], edge_index=edges[f'edge{i}'], edge_attr=edges_attr[f'edge_attr{i}'], y=labels[f'label{i}'])) 

    testlst2=[]
    for i in range(1):
        testlst2.append(Data(x=testnodes2[f'node{i}'], edge_index=testedges2[f'edge{i}'], edge_attr=testedges_attr2[f'edge_attr{i}'],y=testlabels2[f'label{i}']))  
    
    return lst, testlst2
lst, testlst2 = uselst()

In [ ]:
#데이터 로드 및 배치 구성
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader, NeighborLoader

test_loader2 = DataLoader(testlst2, batch_size=1, shuffle=False)
train_loader = NeighborLoader(Batch.from_data_list(lst), batch_size=256, shuffle=True, num_neighbors=[20, 15], num_workers=2, persistent_workers=True)

# 모델 커스터마이징

In [ ]:
import torch
torch.manual_seed(-1)
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv,GATConv,GATv2Conv
import torch.nn as nn
from sklearn.metrics import f1_score, accuracy_score, recall_score,precision_score
from torch_geometric.nn import GIN, GAT
cfg = config(1)

#모델 정의 및 준비
class customSAGEGAT(torch.nn.Module):
    
    def __init__(self, in_channels, hidden_channels,out_channels, heads=8):  #레이어 구성 요소
        super(customSAGEGAT,self).__init__()
        self.sage1 = SAGEConv(in_channels,hidden_channels)
        self.gat1 = GATv2Conv(hidden_channels,hidden_channels, heads=8, edge_dim=1)
        self.sage2 = SAGEConv(hidden_channels*heads, hidden_channels)
        self.sage3 = SAGEConv(hidden_channels,out_channels)

       
    def forward(self,data):  #모델 순방향 과정
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        
        x = self.sage1(x, edge_index)
        x = F.elu(x)
        x= F.dropout(x, training=self.training)

        x= self.gat1(x = x, edge_index = edge_index, edge_attr = edge_attr)
        x = F.elu(x)
        x= F.dropout(x, p=0.6,training=self.training)
        
        x = self.sage2(x, edge_index)
        x = F.elu(x)
        x= F.dropout(x,  p=0.6,training=self.training)

        x = self.sage3(x, edge_index)
        
        return x
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = customSAGEGAT(in_channels=cfg.in_dim,hidden_channels=128,out_channels=1).to(device)
model

# Train

In [ ]:
#학습 함수 정의
def train(loader):
    model.train()
    total_loss = 0
    
    for i,data in enumerate(loader,0): 
        
        data = data.to(device)
        optimizer.zero_grad()
        
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss/ len(loader)
    
#테스트 함수 정의
def test(loader):
    model.eval()
    predict = list()
    actual = list()
    logit=list()
    total_loss = 0

    with torch.no_grad():
        for _, data in enumerate(loader,0):
            data = data.to(device)
            out = model(data)
            loss2 = criterion(out, data.y)
            total_loss += loss2.item()

            m = nn.Sigmoid()   #시그모이드 함수를 적용하여 logit을 0~1 사이의 값으로 변환
            preds = (m(out)>0.5).float().cpu().numpy()
            y = data.y.cpu().numpy()
            logit_batch = m(out).cpu().numpy()
            
            predict.append(preds)
            logit.append(logit_batch)
            actual.append(y)
    
    #평가 지표 계산산
    predictions, actuals = vstack(predict), vstack(actual)
    acc = accuracy_score(actuals, predictions)
    f1  = f1_score(actuals, predictions)
    rec  = recall_score(actuals, predictions, zero_division=1)
    prec  = precision_score(actuals, predictions, zero_division=1)

    return total_loss / len(loader), acc, f1, rec, prec, logit, actual, predict

In [ ]:
from tqdm import tqdm
from numpy import vstack

#손실 함수 및 옵티마이저 설정
criterion = torch.nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

#변수 초기화
best_acc = 0
best_epoch = 0
best_f1 = 0
best_recall = 0
best_precision = 0

#early stopping 설정
early_stopping_epochs = 30
early_stop_counter = 0
best_loss = float('inf')

#학습 루프 실행
for epoch in tqdm(range(80)):
    loss = train(train_loader)
    print(loss)
    torch.save(model.state_dict(), f'/home/work/lib_data/nghl/kisti/model_save/full_0910_classification.pth')  # 모델 가중치 저장

# 추론

In [ ]:
model

In [ ]:
model.load_state_dict(torch.load('/home/work/lib_data/nghl/kisti/model_save/full_0910_classification.pth'))

In [ ]:
train_node = pd.read_csv('/home/work/lib_data/nghl/kisti/data/0910/240910_18_full_node(3년후).csv')
test_node = pd.read_csv('/home/work/lib_data/nghl/kisti/data/0910/240910_19_full_node(3년후).csv')

In [ ]:
# 차년도 추론
for epoch in tqdm(range(1)):

    loss2, acc, f1, rec, prec, logit, actual, predict  = test(test_loader2)

    print(f' Test loss: {loss2:.3f} | Test acc: {acc:.3f} | Test f1: {f1:.3f} | Test rec: {rec:.3f} | Test prec : {prec:.3f} ')

In [ ]:
pd.DataFrame({'acc': [acc],'f1':[f1],'recall':[rec],'precision':[prec]}).transpose()

In [ ]:
len(logit[0])

In [ ]:
#logit값을 붙인 데이터 저장
combined_df_test = pd.concat([pd.DataFrame(logit[0],columns=['logit']),test_node],axis=1)